In [3]:
import torch
from torch import nn

if torch.cuda.is_available():
    print('CUDA is available')
    cuda0 = torch.device('cuda:0')

CUDA is available


tensor([[[[ 1.,  2.,  3.],
          [ 4.,  5.,  6.]],

         [[ 5.,  6.,  7.],
          [ 8.,  9., 10.]]],


        [[[ 9., 10., 11.],
          [12., 12., 14.]],

         [[ 1.,  2.,  3.],
          [ 4.,  5.,  6.]]],


        [[[ 1.,  2.,  3.],
          [ 4.,  5.,  6.]],

         [[ 1.,  2.,  3.],
          [ 4.,  5.,  6.]]]], device='cuda:0', dtype=torch.float64)

tensor([[[[  142.,   188.,   234.],
          [  340.,   449.,   558.]],

         [[ 1862.,  2136.,  2410.],
          [ 2780.,  3189.,  3598.]]],


        [[[ 7302.,  7604.,  8670.],
          [ 9246.,  9628., 10978.]],

         [[  142.,   188.,   234.],
          [  340.,   449.,   558.]]],


        [[[  142.,   188.,   234.],
          [  340.,   449.,   558.]],

         [[  142.,   188.,   234.],
          [  340.,   449.,   558.]]]], device='cuda:0', dtype=torch.float64)

tensor([[[  142.,   188.,   234.,   340.,   449.,   558.],
         [ 1862.,  2136.,  2410.,  2780.,  3189.,  3598.],
         [ 7302.,  7604.,  8670.,  9246.,  9628., 10978.]],

        [[  142.,   188.,   234.,   340.,   449.,   558.],
         [  142.,   188.,   234.,   340.,   449.,   558.],
         [  142.,   188.,   234.,   340.,   449.,   558.]]], device='cuda:0',
       dtype=torch.float64)